In [1]:
import os, torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from collections import OrderedDict
from torch.autograd import Variable

In [2]:
device = torch.device("cuda:0")

In [3]:
class RobertaClassifier(nn.Module):
    def __init__(self, base_model, num_classes, dr_rate, hidden_size):
        super().__init__()

        self.dr_rate = dr_rate
        self.roberta = base_model.roberta
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.roberta_classifier = base_model.classifier
        self.dense = nn.Linear(hidden_size, 384)
        # self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dr_rate)
        self.classifier = nn.Linear(384 , num_classes)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):

        h_0 = Variable(torch.zeros(1, len(input_ids), hidden_size).cuda())
        c_0 = Variable(torch.zeros(1, len(input_ids), hidden_size).cuda())

        out = self.roberta(input_ids=input_ids, attention_mask=attention_mask)[0]
        out, (h, c) = self.lstm(out, (h_0, c_0))
        out = self.roberta_classifier(out)
        out = self.dense(out)
        # out = self.relu(out)
        out = self.dropout(out)
        out = self.classifier(out)
        # out = self.softmax(out)

        return out

In [9]:
def load_dict(file_path, model):

    best_model = torch.load(file_path)

    label = best_model['label']
    pretrained_dict = best_model['state_dict']
    renamed_pretrained_dict = OrderedDict([(k.replace('roberta.', ''), v) for k, v in pretrained_dict.items()])

    model_dict = model.state_dict()

    model_dict.update(renamed_pretrained_dict)
    model.load_state_dict(model_dict)

    return model.to(device), label

In [21]:
def inference(sentence, model):

    attention_mask = sentence['attention_mask'].to(device)
    input_ids = sentence['input_ids'].squeeze(1).to(device)

    out = model(input_ids, attention_mask) # [0]
    max_vals, max_indices = torch.max(out, 1)

    return max_indices

In [6]:
MODEL_NAME = 'klue/roberta-small'
hidden_size = 768

roberta_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=hidden_size)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [37]:
file_path = '/content/drive/MyDrive/Pal-ette/trained_models'
file_name = 'roberta-small_7-emotions.pt'
roberta_model = roberta_model
model = RobertaClassifier(roberta_model, 7, 0.2, hidden_size)
model, label = load_dict(os.path.join(file_path, file_name), model)

In [38]:
text = '아저씨 콩나물도 좀 주세요  네 알았습니다 '

x = tokenizer(text, truncation=True, padding='max_length', max_length=64, return_tensors='pt')
out, p_idx = inference(x, model)
print(label[p_idx])